In [102]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import ast
from apscheduler.schedulers.background import BackgroundScheduler

### Write basic template

In [89]:
# writes the basic template to csv file
# the template contains all the players playing in the EPL in that season, with  playing time, and goals scored, goals lost and plus-minus analysis = 0;

def write_basic_template(source_path, season):
    dest_df = pd.DataFrame(columns=["Season", "Team", "Player Name", "Accumulated Playing Time", "Accumulated Goals Scored", "Accumulated Goals Lost", "Plus-Minus", "Plus-Minus Per Min"])
    source_df = pd.read_csv(source_path)
#     Only obtain the year that you want
    source_df = source_df[source_df["Season"] == season]
#     Only obtain matches with unique home teams
    source_df = source_df.drop_duplicates("Home Team", keep = "first")
    
    for index, row in source_df.iterrows():
        team = row["Home Team"]
        starting_lineup = ast.literal_eval(row["Home Starting Lineup"])
        bench_lineup = ast.literal_eval(row["Home Bench Lineup"])
        players = starting_lineup + bench_lineup
#         Create a row for each player
        for player in players:
            dest_df = dest_df.append({"Season": season, "Team": team, "Player Name": player, "Accumulated Playing Time": 0, "Accumulated Goals Scored": 0, "Accumulated Goals Lost": 0, "Plus-Minus": 0, "Plus-Minus Per Min": 0}, ignore_index = True)
    
#     Write df to csv
    dest_df.to_csv(f"{season}.csv", index = False)
#     Check the number of teams
    dest_df.drop_duplicates("Team", keep = "first").info()
    return dest_df

    
# write_basic_template("match_data_test copy.csv", "2016-2017")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20 entries, 0 to 342
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Season                    20 non-null     object
 1   Team                      20 non-null     object
 2   Player Name               20 non-null     object
 3   Accumulated Playing Time  20 non-null     object
 4   Accumulated Goals Scored  20 non-null     object
 5   Accumulated Goals Lost    20 non-null     object
 6   Plus-Minus                20 non-null     object
 7   Plus-Minus Per Min        20 non-null     object
dtypes: object(8)
memory usage: 1.4+ KB


,Season,Team,Player Name,Accumulated Playing Time,Accumulated Goals Scored,Accumulated Goals Lost,Plus-Minus,Plus-Minus Per Min
0,2016-2017,Hull City,Eldin Jakupović,0,0,0,0,0
1,2016-2017,Hull City,Andrew Robertson,0,0,0,0,0
2,2016-2017,Hull City,Curtis Davies,0,0,0,0,0
3,2016-2017,Hull City,David Meyler,0,0,0,0,0
4,2016-2017,Hull City,Tom Huddlestone,0,0,0,0,0
...,...,...,...,...,...,...,...,...
355,2016-2017,Liverpool,Marko Grujić,0,0,0,0,0
356,2016-2017,Liverpool,Ragnar Klavan,0,0,0,0,0
357,2016-2017,Liverpool,Alberto Moreno,0,0,0,0,0
358,2016-2017,Liverpool,Divock Origi,0,0,0,0,0


### Collate indiv player stats in the specified season

In [103]:
# Obtain the playing time stats for each player
def obtain_playing_time_stats(side, row):
    df = pd.DataFrame(columns=["Season", "Side", "Team", "Player Name", "Playing Start Time", "Playing End Time", "Playing Time", "Goals Scored", "Goals Lost"])
    
    season = row["Season"]
    if side == "Home":
        team = row["Home Team"]
        starting_lineup = ast.literal_eval(row["Home Starting Lineup"])
        sub_lineup = ast.literal_eval(row["Home Sub Timings"])
    elif side == "Away":
        team = row["Away Team"]
        starting_lineup = ast.literal_eval(row["Away Starting Lineup"])
        sub_lineup = ast.literal_eval(row["Away Sub Timings"])

#     Create rows for starting players
    for starting_player in starting_lineup:
        df = df.append({"Season": season, "Side": side, "Team": team, "Player Name": starting_player, "Playing Start Time": 0, "Playing End Time": 90}, ignore_index = True)

#     Set the start time and end time based on the substitutes
#     Players who did not play for this match will have a start and end time of NaN
    for sub in sub_lineup:
        sub_timing = sub[0]
        sub_out_player = sub[1]
        sub_in_player = sub[2]
        
#         No players sub in
        if (sub_in_player == "-"):
            df.loc[df["Player Name"] == sub_out_player, "Playing End Time"] = sub_timing
            df.loc[df["Player Name"] == sub_out_player, "Playing Time"] = sub_timing            
#         Have players sub in
        else:
            df.loc[df["Player Name"] == sub_out_player, "Playing End Time"] = sub_timing
            df = df.append({"Season": season, "Side": side, "Team": team, "Player Name": sub_in_player, "Playing Start Time": sub_timing, "Playing End Time": 90}, ignore_index = True)
    
#     Update the playing time for each player
    for index, row in df.iterrows():
        start_time = row["Playing Start Time"]
        end_time = row["Playing End Time"]
        row["Playing Time"] = end_time - start_time 
    
    return df

# Obtain the goal stats for each player
def obtain_goal_stats(df, home_goal_timings, away_goal_timings, min_playing_time):
    df = df[df["Playing Time"] > min_playing_time]
    
    for index, row in df.iterrows():
        start_time = row["Playing Start Time"]
        end_time = row["Playing End Time"]
        if (start_time == 0 and end_time == 90):
            df.loc[index, "Goals Scored"] = len(home_goal_timings) if (row["Side"]=="Home") else len(away_goal_timings)
            df.loc[index, "Goals Lost"] = len(away_goal_timings) if (row["Side"]=="Home") else len(home_goal_timings)
        else:
            if (row["Side"] == "Home"):
                goal_scored = 0
                goal_lost = 0
                for timing in home_goal_timings:
                    if (timing >= start_time and timing<= end_time):
                        goal_scored +=1
                for timing in away_goal_timings:
                    if (timing >= start_time and timing<= end_time):
                        goal_lost +=1
                df.loc[index, "Goals Scored"] = goal_scored
                df.loc[index, "Goals Lost"] = goal_lost
                
            elif (row["Side"] == "Away"):
                goal_scored = 0
                goal_lost = 0
                for timing in away_goal_timings:
                    if (timing >= start_time and timing<= end_time):
                        goal_scored +=1
                for timing in home_goal_timings:
                    if (timing >= start_time and timing<= end_time):
                        goal_lost +=1
                df.loc[index, "Goals Scored"] = goal_scored
                df.loc[index, "Goals Lost"] = goal_lost

    return df

# obtain stats for players involved in each game
# min_playing_time refers to the min time that the player must play for that match for the stats to be valid
def obtain_indiv_match_stats(row, min_playing_time):
    home_goal_timings = ast.literal_eval(row["Home Goal Timings"])
    away_goal_timings = ast.literal_eval(row["Away Goal Timings"])
    
#     intermediate_df = pd.DataFrame(columns=["Season", "Side", "Team", "Player Name", "Playing Start Time", "Playing End Time", "Goals Scored", "Goals Lost"])
    intermediate_df_1 = obtain_playing_time_stats("Home", row)
    intermediate_df_2 = obtain_playing_time_stats("Away", row)
    intermediate_df = pd.concat([intermediate_df_1, intermediate_df_2], ignore_index = True)
    intermediate_df = obtain_goal_stats(intermediate_df, home_goal_timings, away_goal_timings, min_playing_time)
        
    return intermediate_df
# obtain_indiv_match_stats(row, 10)

In [105]:
# remove players who did not play at all this season
def remove_non_playing_players(df, min_playing_time_per_season):
    df = df[df["Accumulated Playing Time"] > min_playing_time_per_season]
    return df

def plus_minus_analysis(source_path, season, min_playing_time_per_game, min_playing_time_per_season):
    source_df = pd.read_csv(source_path)
    dest_path = f"{season}.csv"
    dest_df = pd.read_csv(dest_path)
    season_df = source_df[source_df["Season"] == season]
    for season_index, season_row in season_df.iterrows():
        intermediate_df = obtain_indiv_match_stats(season_row, min_playing_time_per_game)
        for intermediate_index, intermediate_row in intermediate_df.iterrows():
            player_name = intermediate_row["Player Name"]
            playing_time = intermediate_row["Playing Time"]
            goal_scored = intermediate_row["Goals Scored"]
            goal_lost = intermediate_row["Goals Lost"]
            dest_df.loc[dest_df["Player Name"] == player_name, "Accumulated Playing Time"] += playing_time
            dest_df.loc[dest_df["Player Name"] == player_name, "Accumulated Goals Scored"] += goal_scored
            dest_df.loc[dest_df["Player Name"] == player_name, "Accumulated Goals Lost"] += goal_lost
            
#     update the "Plus-Minus" and "Plus-Minus Per Min"
    for dest_index, dest_row in dest_df.iterrows():
        plus_minus = int(dest_row["Accumulated Goals Scored"]) - int(dest_row["Accumulated Goals Lost"])
        playing_time = int(dest_row["Accumulated Playing Time"])
        dest_df.loc[dest_index, "Plus-Minus"] = plus_minus
        if playing_time > 0:   
            dest_df.loc[dest_index, "Plus-Minus Per Min"] = plus_minus/playing_time

#     remove players who did not play at all this season
    dest_df = remove_non_playing_players(dest_df, min_playing_time_per_season)
#     write df to csv
    dest_df.to_csv(dest_path, index = False)

    return dest_df

In [107]:
# seasons = ["2020-2021", "2019-2020", "2018-2019", "2017-2018", "2016-2017"]
seasons = ["2021-2022"]

for s in seasons:
    write_basic_template("match_data_test copy.csv", s)
    plus_minus_analysis("match_data_test copy.csv", s, 10, 450)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20 entries, 0 to 379
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Season                    20 non-null     object
 1   Team                      20 non-null     object
 2   Player Name               20 non-null     object
 3   Accumulated Playing Time  20 non-null     object
 4   Accumulated Goals Scored  20 non-null     object
 5   Accumulated Goals Lost    20 non-null     object
 6   Plus-Minus                20 non-null     object
 7   Plus-Minus Per Min        20 non-null     object
dtypes: object(8)
memory usage: 1.4+ KB


/Users/kaijun/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/kaijun/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)
